#  Assignment 2 - Transfer Learning and Data Augmentation 💬

Welcome to the **second assignment** for the **CS-552: Modern NLP course**!

> - 😀 Name: **Mathieu Desponds**
> - ✉️ Email: **mathieu.desponds@epfl.ch**
> - 🪪 SCIPER: **283229**

<div style="padding:15px 20px 20px 20px;border-left:3px solid green;background-color:#e4fae4;border-radius: 20px;">

## **Assignment Description**
- In the first part of this assignment, you will need to implement training (fine-tuning) and evaluation of a pre-trained language model ([DistilBERT](https://huggingface.co/docs/transformers/model_doc/distilbert) ), on natural language inference (NLI) task for recognizing textual entailment (RTE).

- Following the first finetuning task, you will need to identify the shortcut (i.e. some salient or toxic features) that the model learnt for the specific task. 

- For part-3, you are supposed to annotate 100 randomly assigned test datapoints as ground-truth labels. Additionally, the cross annotation should be conducted by another one or two annotators, and you will learn about how to calculate the agreement statistics as a significant characteristic reflecting the quality of a collected dataset.

- For part-4, since the human annotation is quite time- and effort-consuming, there are plenty of ways to get silver-labels from automatic labeling to augment the dataset scale. We provide the reference to some simple methods (EDA and Back Translation) but you are encouraged to explore other advanced mechanisms. You will evaluate the improvement of your model performance by using your data augmentation method.

For each part, you will need to complete the code in the corresponding `.py` files (`nli.py` for Part-1, `shortcut.py` for Part-2, `eda.py` for Part-4). You will be provided with the function descriptions and detailed instructions about the code snippet you need to write.


### Table of Contents
- **[PART 1: Model Finetuning for NLI](#1)**
    - [1.1 Data Processing](#11)
    - [1.2 Model Training and Evaluation](#12)
- **[PART 2: Identify Model Shortcut](#2)**
    - [2.1 Word-Pair Pattern Extraction](#21)
    - [2.2 Distill Potentially Useful Patterns](#22)
    - [2.3 Case Study](#23)
- **[PART 3: Annotate New Data](#3)**
    - [3.1 Write an Annotation Guideline](#31)
    - [3.2 Annotate Your 100 Datapoints with Partner(s)](#32)
    - [3.3 Agreement Measure](#33)
    - [3.4 Robustness Check](#34)
- **[PART 4: Data Augmentation](#4)**
    
### Deliverables

- ✅ This jupyter notebook
- ✅ `nli.py` file
- ✅ `shortcut.py` file
- ✅ Finetuned DistilBERT models for NLI task (Part 1 and Part 4)
- ✅ Annotated and cross-annotated data files (Part 3)
- ✅ New dataset from data augmentation (Part 4)

</div>

### Google Colab Setup
If you are using Google Colab notebook for this assignment, you will need to run a few commands to set up our environment on Google Colab. If you are running this notebook on a local machine you can skip this section.

Run the following cell to mount your Google Drive. Follow the popped window, sign in to your Google account. (The same account you used to store this notebook!)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Now first click the 4th left-side bar (named Files), then click the 2nd bar popped under Files column (named Refresh), under "/drive/MyDrive/" find the Assignment 2 folder that you uploaded to your Google Drive, copy its path and fill it in below. If everything is working correctly, then running the folowing cell should print the filenames from the assignment:

```
['Assignment2.ipynb', 'requirements.txt', 'runs', 'predictions', 'nli_data', 'testA2.py', 'nli.py', 'shortcut.py']
```

In [ ]:
import os
# TODO: Fill in the path where you download the Assignment folder into
ROOT_PATH = "./drive/MyDrive/A2" # Replace with your directory to A2 folder
print(os.listdir(ROOT_PATH))

Before we start, we also need to run some boilerplate code to set up our environment, same as previous assignments. You'll need to rerun this setup code each time you start the notebook.

In [ ]:
requirements = ROOT_PATH + "/requirements.txt"
!pip install torch==1.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install -r {requirements}


Run this cell to load the autoreload extension. This allows us to edit .py source files, and re-import them into the notebook for a seamless editing and debugging experience.

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from copy import deepcopy
import numpy as np 
from tqdm import tqdm
import jsonlines
import sys
import time
import random

import torch
import torch.utils.data
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_constant_schedule_with_warmup
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

Once you have successfully mounted your Google Drive and located the path to this assignment, run the following cell to allow us to import from the `.py` files of this assignment. If it works correctly, it should print the message:

```
Hello A2!
```

In [9]:
sys.path.append(ROOT_PATH)

from testA2 import hello_A2
hello_A2()

Hello A2!


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Note that if CUDA is not enabled, `torch.cuda.is_available()` will return False and this notebook will fallback to CPU mode.

The global variables `dtype` and `device` will control the data types throughout this assignment.

We will be using `torch.float = torch.float32` for all operations.

Please refer to https://pytorch.org/docs/stable/tensor_attributes.html#torch-dtype for more details about data types.

In [10]:
if torch.cuda.is_available():
    print('Good to go!')
else:
    print('Please set GPU via Edit -> Notebook Settings.')

Good to go!


### Local Setup
If you skip Google Colab setup, you still need to fill in the path where you download the Assignment folder, and install required packages.

In [2]:
ROOT_PATH = "./" # Replace with your directory to A2 folder

In [ ]:
requirements = ROOT_PATH + "/requirements.txt"
!pip install -r {requirements}

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from copy import deepcopy
import numpy as np 
from tqdm import tqdm
import jsonlines
import sys
import time, os
import random

import torch
import torch.utils.data
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_constant_schedule_with_warmup
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

<a name="1"></a>
## **PART 1: Finetuning DistilBERT for NLI**
---

### **What is the NLI task?🧐**
> Given a pair of sentences, denoted as a "premise" sentence and a "hypothesis" sentence, NLI (or RTE) aims to determine their logical relationship, i.e. whether they are logically follow (entailment), unfollow (contradiction) or are undetermined (neutral) to each other.

> Defined as a machine learning task, NLI can be considered as a 3-classes (entailment, contradiction, or neutral) classification task, with a sentence-pair input ("hypothesis" and “premise”).

> **You can run the following cell to have the first glance at your data**. Each data sample is a python dictionary, which consists of following components:
- premise sentence (*'premise'*), 
- hypothesis sentence (*'hypothesis'*) 
- domain (*'domain'*): describing the topic of premise and hypothesis sentences (e.g., government regulations, telephone talks, etc.)
- label (*'label'*): indicating the logical relation between premise and hypothesis (i.e., entailment, contradiction, or neutral).

In [6]:
# If you use Google Colab, then data_dir = 'GOOGLE_DRIVE_PATH/nli_data'
data_dir = ROOT_PATH+'/nli_data'
data_dev_path = os.path.join(data_dir, 'dev_in_domain.jsonl')
with jsonlines.open(data_dev_path, "r") as reader:
    for sid, sample in enumerate(reader.iter()):
        print(sample)
        if sid == 2:
            break

{'premise': 'The new rights are nice enough', 'hypothesis': 'Everyone really likes the newest benefits ', 'domain': 'slate', 'label': 'neutral'}
{'premise': 'This site includes a list of all award winners and a searchable database of Government Executive articles.', 'hypothesis': 'The Government Executive articles housed on the website are not able to be searched.', 'domain': 'government', 'label': 'contradiction'}
{'premise': "uh i don't know i i have mixed emotions about him uh sometimes i like him but at the same times i love to see somebody beat him", 'hypothesis': 'I like him for the most part, but would still enjoy seeing someone beat him.', 'domain': 'telephone', 'label': 'entailment'}


In [7]:
# Enter enter your Sciper number
SCIPER = '283229'
seed = int(SCIPER)
torch.backends.cudnn.deterministic = True

In [8]:
print('Your random seed is: ', seed)

Your random seed is:  283229


In [9]:
# We use the following pretrained tokenizer and model
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=3)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifi

### **1.1 Dataset Processing**
Our first step is to load datasets for NLI task by constructing a Pytorch Dataset. Specifically, we will need to implement tokenization and padding with a HuggingFace pre-trained tokenizer.

**Complete `NLIDataset` class following the instructions in `nli.py`, and test by running the following cell.**

In [10]:
from nli import NLIDataset
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
dataset = NLIDataset(ROOT_PATH+"/nli_data/dev_in_domain.jsonl", tokenizer)

Building NLI Dataset...


9815it [00:09, 1041.06it/s]


In [11]:
from testA2 import test_NLIDataset
test_NLIDataset(dataset)

NLIDataset test correct ✅


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/desponds/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### **1.2 Model Training and Evaluation**
Next, we will implement the training and evaluation process to finetune the model. For model training, you will need to calculate the loss and update the model weights by update the optimizer. Additionally, we add a learning rate schedular to adopt an adaptive learning rate during the whole training process. 

For evaluation, you will need to compute accuracy and F1 scores to assess the model performance. 

**Complete the `compute_metric()`, `train()` and `evaluate()` functions following the instructions in the `nli.py` file, you can test compute_metric() by running the following cell.**

In [12]:
from nli import compute_metrics, train, evaluate

from testA2 import test_compute_metrics
test_compute_metrics(compute_metrics)

compute_metric test correct ✅


#### **Start Training and Validation!**

Try the following different hyperparameter settings, compare and discuss the results. (Other hyperparameters should not be changed.)

> A. learning_rate 2e-5

> B. learning_rate 5e-5

**Note:** *Each training will take about 1 hour using a GPU, please keep your computer and notebook active during the training.*

**Questions: Which learning rate is better? Explain your answers.**

In [33]:
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=3)
model.to(device)

train_dataset = NLIDataset(ROOT_PATH+"/nli_data/train.jsonl", tokenizer)
dev_dataset = NLIDataset(ROOT_PATH+"/nli_data/dev_in_domain.jsonl", tokenizer)

batch_size = 16
epochs = 4
max_grad_norm = 1.0
warmup_percent = 0.3
model_save_root = ROOT_PATH+'/runs/'

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

Building NLI Dataset...


98176it [01:38, 999.55it/s] 


Building NLI Dataset...


9815it [00:10, 980.48it/s]


In [ ]:
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

learning_rate = 2e-5 # play around with this hyperparameter

train(train_dataset, dev_dataset, model, device, batch_size, epochs,
      learning_rate, warmup_percent, max_grad_norm, model_save_root)

Evaluation: 100%|██████████| 614/614 [00:23<00:00, 26.66it/s]


Epoch: 0 | Training Loss: 0.842 | Validation Loss: 0.660
Epoch 0 NLI Validation:
Accuracy: 72.83% | F1: (75.21%, 69.40%, 73.97%) | Macro-F1: 72.86%
Model Saved!


Evaluation: 100%|██████████| 614/614 [00:23<00:00, 25.97it/s]


Epoch: 1 | Training Loss: 0.567 | Validation Loss: 0.602
Epoch 1 NLI Validation:
Accuracy: 77.08% | F1: (79.58%, 73.21%, 78.25%) | Macro-F1: 77.01%
Model Saved!


Evaluation: 100%|██████████| 614/614 [00:23<00:00, 26.53it/s]


Epoch: 2 | Training Loss: 0.329 | Validation Loss: 0.633
Epoch 2 NLI Validation:
Accuracy: 76.70% | F1: (80.23%, 71.85%, 77.47%) | Macro-F1: 76.52%


Evaluation: 100%|██████████| 614/614 [00:23<00:00, 26.40it/s]

Epoch: 3 | Training Loss: 0.182 | Validation Loss: 0.909
Epoch 3 NLI Validation:
Accuracy: 76.34% | F1: (80.15%, 71.26%, 76.96%) | Macro-F1: 76.12%


In [ ]:
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

learning_rate = 5e-5 # play around with this hyperparameter

train(train_dataset, dev_dataset, model, device, batch_size, epochs,
      learning_rate, warmup_percent, max_grad_norm, model_save_root)

Evaluation: 100%|██████████| 614/614 [00:24<00:00, 24.63it/s]


Epoch: 0 | Training Loss: 0.365 | Validation Loss: 0.785
Epoch 0 NLI Validation:
Accuracy: 74.53% | F1: (75.99%, 71.69%, 76.23%) | Macro-F1: 74.64%
Model Saved!


Evaluation: 100%|██████████| 614/614 [00:24<00:00, 24.69it/s]


Epoch: 1 | Training Loss: 0.371 | Validation Loss: 0.820
Epoch 1 NLI Validation:
Accuracy: 74.90% | F1: (78.53%, 69.15%, 76.25%) | Macro-F1: 74.64%
Model Saved!


Evaluation: 100%|██████████| 614/614 [00:24<00:00, 24.61it/s]


Epoch: 2 | Training Loss: 0.281 | Validation Loss: 0.703
Epoch 2 NLI Validation:
Accuracy: 74.81% | F1: (78.49%, 69.88%, 75.51%) | Macro-F1: 74.62%


Evaluation: 100%|██████████| 614/614 [00:24<00:00, 24.71it/s]

Epoch: 3 | Training Loss: 0.212 | Validation Loss: 0.866
Epoch 3 NLI Validation:
Accuracy: 74.66% | F1: (78.35%, 69.33%, 75.64%) | Macro-F1: 74.44%


**Answer** : Based on the results of the two training sessions, it appears that using `learning_rate = 2e-5` yields superior outcomes compared to using `learning_rate = 5e-5`. The initial progress of the first model was comparatively slower and it reached its maximum later than the second model. On the other hand, the second model showed quicker progress in the beginning due to the higher learning rate, but ultimately failed to converge as the large learning step prevent its progress towards the maximum.

### **Fine-Grained Validation**

Use the model checkpoint saved under the first hyperparameter setting (learning_rate 2e-5) in 1.4, check the model performance on each domain subsets of the validation set, report the validation loss, accuracy, F1 scores and Macro-F1 on each domain, compare and discuss the results.

**Questions: On which domain does the model perform the best? the worst? Give some possible explanations of why the model's best-performed domain is easier, and why the model's worst-performed domain is more challenging. Use some examples to support your explanations.**

**Note:** To find examples for supporting your discussion, save the model prediction results on each domain under the './predictions/' folder, by specifying the *result_save_file* of the *evaluate* function.

In [19]:
batch_size = 16
learning_rate = 2e-5
warmup_percent = 0.3
checkpoint = ROOT_PATH+'/runs/lr{}-warmup{}'.format(learning_rate, warmup_percent)

# Split the validation sets into subsets with different domains
# Save the subsets under './nli_data/'
# Replace "..." with your code

In [ ]:
data_by_domain = {}
with jsonlines.open(data_dev_path, "r") as reader:
    for sid, sample in enumerate(reader.iter()):
        if sample['domain'] in data_by_domain :
            data_by_domain[sample['domain']].append(sample)
        else :
            data_by_domain[sample['domain']] = [sample]

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer = DistilBertTokenizer.from_pretrained(checkpoint)
model = DistilBertForSequenceClassification.from_pretrained(checkpoint)
model.to(device)

for domain in ["fiction", "government", "slate", "telephone", "travel"]:
    
    # Evaluate and save prediction results in each domain
    # Replace "..." with your code
    path = f"{ROOT_PATH}/nli_data/domain_{domain}.jsonl"
    with jsonlines.open(path, mode='w') as writer:
        for sample in data_by_domain[domain]:
            writer.write(sample)
    dev_domain_dataset = NLIDataset(path, tokenizer)
    dev_loss, acc, f1_ent, f1_neu, f1_con = evaluate(
        dev_domain_dataset, model, device, batch_size, no_labels=False, result_save_file= f"{ROOT_PATH}/predictions/domain_{domain}.jsonl")
    macro_f1 = (f1_ent + f1_neu + f1_con) / 3
    
    print(f'Domain: {domain}')
    print(f'Validation Loss: {dev_loss:.3f} | Accuracy: {acc*100:.2f}%')
    print(f'F1: ({f1_ent*100:.2f}%, {f1_neu*100:.2f}%, {f1_con*100:.2f}%) | Macro-F1: {macro_f1*100:.2f}%')

Building NLI Dataset...


1973it [00:01, 1670.85it/s]
Evaluation: 100%|██████████| 124/124 [00:03<00:00, 32.67it/s]


Domain: fiction
Validation Loss: 0.605 | Accuracy: 77.14%
F1: (79.45%, 72.35%, 79.05%) | Macro-F1: 76.95%
Building NLI Dataset...


1945it [00:03, 581.01it/s]
Evaluation: 100%|██████████| 122/122 [00:05<00:00, 23.31it/s]


Domain: government
Validation Loss: 0.477 | Accuracy: 81.65%
F1: (84.16%, 77.77%, 82.69%) | Macro-F1: 81.54%
Building NLI Dataset...


...

Domain: slate
Validation Loss: 0.698 | Accuracy: 72.07%
F1: (74.46%, 67.79%, 73.68%) | Macro-F1: 71.98%
Building NLI Dataset...


1966it [00:02, 669.86it/s]
Evaluation: 100%|██████████| 123/123 [00:06<00:00, 20.43it/s]


Domain: telephone
Validation Loss: 0.595 | Accuracy: 76.96%
F1: (80.03%, 71.73%, 78.65%) | Macro-F1: 76.80%
Building NLI Dataset...


1976it [00:01, 1106.10it/s]
Evaluation: 100%|██████████| 124/124 [00:04<00:00, 25.17it/s]


Domain: travel
Validation Loss: 0.561 | Accuracy: 78.29%
F1: (82.40%, 74.76%, 77.37%) | Macro-F1: 78.18%


In [70]:
save_repo_gvt = f"{ROOT_PATH}/predictions/domain_government.jsonl"
save_repo_slate = f"{ROOT_PATH}/predictions/domain_slate.jsonl"
predictions = []
for i, repo in enumerate(zip([save_repo_gvt, save_repo_slate], ['government', 'slate'])):
    with jsonlines.open(repo[0], "r") as reader:
        print(f"\n***********************************")
        print(f"Exemples of {repo[1]}")
        idx = np.random.randint(0,100, size = 3)
        for i, sample in enumerate(reader.iter()):
            if i in idx :
                print(f"""
    premise : {sample['premise']},
    hypothesis : {sample['hypothesis']}""")



***********************************
Exemples of government

    premise : GAO recommends that the Secretary of Defense revise policy and guidance,
    hypothesis : GAO recommends that the Secretary of Defense revise policy and guidance

    premise : Now suppose there is a private delivery firm in Cleveland that is competing with the postal service.,
    hypothesis : Imagine a state-run delivery firm trying to phase out the postal service.

    premise : The Commission's analysis uses both quantifiable and general descriptions of the effects of the rule on small entities.,
    hypothesis : The rule has a significant effect on small entities.

***********************************
Exemples of slate

    premise : Even the lower limit of that differential compounds to a hefty sum over time.,
    hypothesis : The differential will not grow.

    premise : Tom is the winner of a year's supply of Turtle Wax, and he will receive his prize just as soon as the Shopping Avenger figures out how m

**Answer** : The domain where the model perform the best is `government` and where it perform the worst is `slate`.

By looking deeper at the dataset and at the different domains, we can see that samples in the `government` domain are political or general declarative statment and are mostly not ambiguous. On the other side, `slate` domain is more ambiguous and labeling is harder on this domain. 

## **Task2: Identify Shortcuts**

We aim to find some shortcuts that the model in 1.4 (under the first hyperparameter setting) has learned.

### **2.1 Word-Pair Pattern Extraction**

We consider to exatrct simple word-pair patterns that the model may have learned from the NLI data. 

For this, we assume that a pair of words that occur in a premise-hypothesis sentence pair (one occurs in premise and the other occurs in hypothesis) may serve as a key indicator of the logical relationship between the premise and hypothesis sentences. For example:

>- Premise: Consider the United States Postal Service.
>- Hypothesis: Forget the United States Postal Service.

Here the word-pair "consider" and "forget" determine that the premise and hypothesis have a *contradiction* relationship, so (consider, forget) --> *contradiction* might be a good pattern to learn.

**Note:** 
- We do not consider the naive word pair patterns where the word from premise and the word from hypothesis are identical, e.g., (service, service) got from the above premise-hypothesis sentence pair.
- We do not consider stop words neither, punctuations and words that contain special prefix '##', e.g., '##s' in the pattern extraction.

In [20]:
# stop_words and puntuations to be removed from consideration in the pattern extraction

import nltk
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
stop_words.append('uh')

import string
puncs = string.punctuation

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Complete `word_pair_extraction()` function in `shortcut.py` file.**

The keys of the returned dictionary *word_pairs* should be **different word-pairs** appered in premise-hypothesis sentence pairs, i.e., (a word from the premise, a word from the hypothesis).

The value of a word-pair key records the counts of entailment, neutral and contradiction predictions **made by the model** when the word-pair occurs, i.e., \[#entailment_predictions, #neutral_predictions,  #contradiction_predictions\].

**Note:** Remember to remove naive word pairs (i.e., premise word identical to hypothesis word), stop_words, puntuations and words with special prefix '##' out of consideration.

### **2.2 Distill Potentially Useful Patterns**

Find and print the **top-100** word-pairs that are associated with the **largest total number** of model predictions, which might contain frequently used patterns.

In [17]:
from shortcut import word_pair_extraction

In [43]:
# all your saved model prediction results in 1.2 Fine-Grained Validation
domains = ["fiction", "government", "slate", "telephone", "travel"]
prediction_files = [f"{ROOT_PATH}/nli_data/val_by_domains/domain_{domain}.jsonl" for domain in domains]

tokenizer = DistilBertTokenizer.from_pretrained(checkpoint)
word_pairs = word_pair_extraction(prediction_files, tokenizer)

# find top-100 word-pairs associated with the largest total number of model predictions
top_100_freq_pairs = dict(sorted(word_pairs.items(), key=lambda item: -sum(item[1]))[:100])

print(top_100_freq_pairs)

{('know', 'i'): [28, 36, 23], ('don', 'i'): [27, 22, 24], ('like', 'i'): [16, 18, 12], ('services', 'legal'): [19, 12, 15], ('know', 'don'): [14, 15, 15], ('legal', 'services'): [16, 8, 15], ('yeah', 'i'): [11, 15, 10], ('im', 'i'): [16, 12, 6], ('time', 'i'): [15, 10, 7], ('service', 'postal'): [12, 9, 11], ('go', 'i'): [15, 7, 9], ('would', 'i'): [7, 18, 5], ('one', 'i'): [11, 9, 10], ('really', 'i'): [11, 11, 8], ('think', 'i'): [14, 9, 6], ('know', 'lot'): [8, 19, 2], ('postal', 'service'): [10, 9, 9], ('that', 'i'): [10, 9, 9], ('know', 'make'): [4, 13, 11], ('would', 'could'): [7, 14, 5], ('got', 'i'): [13, 6, 7], ('like', 'don'): [5, 5, 16], ('get', 'i'): [9, 7, 9], ('test', 'toxicity'): [8, 8, 8], ('iv', 'i'): [10, 9, 5], ('um', 'i'): [6, 14, 4], ('like', 'think'): [14, 8, 1], ('know', 'never'): [0, 0, 23], ('know', 'time'): [3, 14, 5], ('see', 'i'): [7, 6, 8], ('could', 'would'): [6, 13, 2], ('ill', 'i'): [10, 1, 9], ('try', 'would'): [7, 1, 12], ('last', 'years'): [11, 5, 4],

**Among the top-100 frequent word-pairs above**, find out the **top-5** word-pairs whose occurances **most likely** lead to *entailment* predictions (entailment patterns), and the **top-5** word-pairs whose occurances **most likely** lead to *contradiction* predictions (contradiction patterns).

**Explain your rules for finding these word pairs.**

Explanation of the rule to find the word-pair whose occurances **most likely** lead to *entailment*:

We want to maximize the probability that a pair of words gives  *entailment*. This can we written as maximizing $ C_e / (C_e + C_n + C_c)$ where $C_e, C_n, C_c$ is the number of times the word pair appears in  *entailment*, *neutral* and  *contradiction* respectively. 


In [22]:
# find top-5 entailment and contradiction patterns
top_5_entailment = dict(sorted(top_100_freq_pairs.items(), key=lambda item: -item[1][0] / sum([i for i in item[1]]))[:5])
top_5_contradict = dict(sorted(top_100_freq_pairs.items(), key=lambda item: -item[1][2] / sum([i for i in item[1]]))[:5])

print("Entailment Patterns:")
print(top_5_entailment)
print("Contradiction Patterns:")
print(top_5_contradict)

Entailment Patterns:
{('information', 'state'): [15, 0, 0], ('don', 'like'): [12, 2, 3], ('may', 'might'): [12, 4, 2], ('know', 'like'): [13, 7, 0], ('many', 'people'): [10, 3, 3]}
Contradiction Patterns:
{('know', 'never'): [0, 0, 23], ('like', 'never'): [3, 1, 16], ('out', 'would'): [6, 0, 12], ('know', 'think'): [5, 1, 11], ('know', 'go'): [3, 3, 11]}


### **2.3 Case Study**

Find out and study **4 representative** cases where the pattern that you have found in 2.2 **fails**, e.g., the premise-hypothesis sentence pair contains ('good', 'bad'), but has an *entailment* gold label.

**Based on your case study, explain the limitations of the word-pair patterns.**

**Answer** : 
To begin with, based on the previous point, it is evident that the frequent occurrence of certain word pairs in a particular context does not necessarily hold significant meaning for humans. It is unexpected that the words "know" and "never" only appear in *conradiction* to each other, indicating a certain degree of limitation of word-pair patterns.

Furthermore, the case study demonstrates that although the words "like" and "never" appear in the premise and hypothesis respectively, the sentences are lengthy and do not exhibit a direct contradiction between the two words. 

In [44]:
representatives = []
study_cases = [('like', 'never'), ('know', 'think')]
for pred_file in prediction_files:
    with jsonlines.open(pred_file, "r") as reader:
        for sample in reader.iter():
            for study_case in study_cases :
                if study_case[0] in sample['premise'] and \
                    study_case[1] in sample['hypothesis'] and \
                    sample['label'] != 'contradiction' :
                    representatives.append(sample)
representatives[:4]

[{'premise': 'I\'m not sentimental, you know." She paused.',
  'hypothesis': "Everyone thinks she's sentimental. ",
  'domain': 'fiction',
  'label': 'neutral'},
 {'premise': "And, although I got a Ph.D. in philosophy many years ago and have thought and read about these matters ever since, heaven (or whatever) knows I don't have too many answers that I feel confident about.",
  'hypothesis': "I have been thinking about this since I earned my degree, but I still don't know for sure.",
  'domain': 'slate',
  'label': 'entailment'},
 {'premise': 'you sound like this girl that i talked to about books and we got into movies one night',
  'hypothesis': 'I found out about so many movies I had never heard of.',
  'domain': 'telephone',
  'label': 'neutral'},
 {'premise': "because i i mean i don't know it's just something i think something we need",
  'hypothesis': 'I think it is something that we need.',
  'domain': 'telephone',
  'label': 'entailment'}]

## **Task3: Annotate New Data**

To check the robustness of developed model, **some additional sets of test data** are collected (under /nli_data/test_data/), which contain NLI samples that are out of the domains of the training and validation data.

However, the test data does not have gold labels of the relationships between premise and hypothesis sentences, i.e., all the labels are marked as *hidden*. **We consider to annotate the data by ourselves.**

### **3.1 Write an Annotation Guideline**

Imagine that you are going to assign this annotation task to a crowdsourcing worker, who is completely not familiar with computer science and NLP. Think about how you are going to explain this annotation task to him in order to guide him do a decent job. Write an annotation guideline for such a worker who are going to do this task for you.

**Note:** You should come up with your own guideline without the help of your partner(s) in later Task 3.2

Annotation guideline: 

**In general** :
- Avoid making assumptions or drawing inferences beyond what is explicitly stated in the *premise* and *hypothesis*.

**Both of the sentences are questions** :
- Classify the sample as *entailment* if the *hypothesis* is a restatement or a paraphrase of the *premise* that preserves the meaning of the original statement.
- Classify the sample as *contradiction* if the *hypothesis* is asking the contrary of the *premise*, i.e., if it contradicts the information presented in the *premise*.
- Classify the sample as neutral if there is no clear link between the *premise* and the *hypothesis* or if the *hypothesis* is unrelated to the *premise*.

**Both of the sentences are NOT questions** :
- Classify the sample as *entailment* if the *hypothesis* can be said to be true only by knowing the *premise*.
- Classify the sample as *contradiction* if the *hypothesis* can be said to be wrong only by knowing the *premise*, or if there is a clear contradiction between the two sentence. e.g. "A cat is on the fridge", "A cat is on the sofa".
- Classify the sample as *neutral* if we can not say if the *hypothesis* is true or false only knowing the *premise*.

### **3.2 Annotate Your 100 Datapoints with Partner(s)**

Annotate your 100 test datapoints with your partner(s), by editing the value of the key "label_student1", "label_student2" and "label_student3" (if you are in a group of three students) in each datapoint.

**Note:** 
- You can download the assigned annotation file (`<your-testset-id>.jsonl`) by [this link](https://drive.google.com/drive/folders/146ExExmpnSUayu6ArGiN5gQzCPJp0myB?usp=share_link)
- Please find your annotation partner according to the "Student Pairing List for A2 Task3" shared on Ed.

**Name your annotated file as `<index>-<sciper_number>.jsonl`.** 

For example, if you get `01.jsonl` to annotate, you should name your deliverable as `01-<your_sciper_number>.jsonl`.

In [ ]:
# Read the json file 
text_samples = []
with jsonlines.open(data_dir+"/12.jsonl", "r") as reader:
    for sample in reader.iter():
        text_samples.append(sample)

# Read the text file where my labels are written and write them in text_samples
labels = []
with open(data_dir+"/labels.txt") as reader :
    labels = [line[0] for line in reader.readlines()]
    
for i, lab in enumerate(labels) :
    if lab == 'E':
        labels[i] = 'entailment'
    elif lab == 'N' :
        labels[i] = 'neutral'
    if lab == 'C':
        labels[i] = 'contradiction'
        
for i, text in enumerate(text_samples):
    text_samples[i].update({'label_student2': labels[i]})

# Write the jsonl file with my labels
with jsonlines.open(data_dir+'/12-283229.jsonl', mode='w') as writer:
    for text in text_samples:
        writer.write(text)

### **3.3 Agreement Measure**

Based on your and your partner's annotations on the 100 test datapoints in 3.2, calculate the [Cohen's Kappa](https://scikit-learn.org/stable/modules/model_evaluation.html#cohen-kappa) or [Krippendorff's Alpha](https://github.com/pln-fing-udelar/fast-krippendorff) (if you are in a group of three students) between the annotators. Discuss the agreement measure results.

**Note:** Cohen's Kappa or Krippendorff's Alpha interpretation

0: No Agreement

0 ~ 0.2: Slight Agreement

0.2 ~ 0.4: Fair Agreement

0.4 ~ 0.6: Moderate Agreement

0.6 ~ 0.8: Substantial Agreement

0.8 ~ 1.0: Near Perfect Agreement

1.0: Perfect Agreement

> **Questions**: What is your interpretation of Cohen's Kappa or Krippendorff's Alpha value according to the above mapping? Which kind of disagreements are most frequently happen between you and your partner(s), i.e., *entailment* vs. *neutral*, *entailment* vs. *contradiction*, or *neutral* vs. *contradiction*? For the second question, give some examples to explain why that is the case. Are there possible ways to address the disagrrements between two annotators?

In [ ]:
# Read the labels of my teammate
with jsonlines.open(data_dir+"/david.jsonl", "r") as reader:
    for i,sample in enumerate(reader.iter()):
        text_samples[i].update({'label_student1': sample['label_student1']})

# Compute the Cohan Kappa Score and the accuracy between us 
from sklearn.metrics import cohen_kappa_score
my_label = [text['label_student2'] for text in text_samples]
his_label = [text['label_student1'] for text in text_samples]
accuracy = sum([1 for i in range(len(my_label)) if my_label[i] ==his_label[i]])/len(my_label)
print(f"The Cohen's Kappa Alpha value is {cohen_kappa_score(my_label, his_label)} which according to the above mapping means a Substantial Agreement.")
print(f"The percentage of aggreament is {accuracy}%")

# Get insight about our disagreements 
e_vs_n = sum([1 for i in range(len(my_label)) if (my_label[i] == "entailment" and his_label[i] == "neutral")])
n_vs_e = sum([1 for i in range(len(my_label)) if (my_label[i] == "neutral" and his_label[i] == "entailment")])
e_vs_c = sum([1 for i in range(len(my_label)) if (my_label[i] == "entailment" and his_label[i] == "contradiction")])
c_vs_e = sum([1 for i in range(len(my_label)) if (my_label[i] == "contradiction" and his_label[i] == "entailment")])
c_vs_n = sum([1 for i in range(len(my_label)) if (my_label[i] == "neutral" and his_label[i] == "contradiction")])
n_vs_c = sum([1 for i in range(len(my_label)) if (my_label[i] == "contradiction" and his_label[i] == "neutral")])
print(f"""Over the {int(100- accuracy*100)} disagreement,
{e_vs_n+n_vs_e} are between entailement and neutral ({e_vs_n},{n_vs_e}) 
{e_vs_c+c_vs_e} are between entailement and contradiction ({e_vs_c},{c_vs_e}), 
{c_vs_n+n_vs_c} are between neutral and contradiction ({c_vs_n},{n_vs_c})
where the parenthesis mean for the first line that :
   (I choose entailment and he said neutral, I choose neutral and he said entailment)""")

Over the 24 disagreement,
12 are between entailement and neutral (5,7) 
9 are between entailement and contradiction (4,5), 
3 are between neutral and contradiction (2,1)
where the parenthesis mean for the first line that :
   (I choose entailment and he said neutral, I choose neutral and he said entailment)


**Answer** : The Cohen's Kappa Alpha value is $0.627$ which according to the above mapping means a Substantial Agreement.

An analysis of the labeled data revealed that the most frequent types of disagreements were between *entailment* and *neutral*, followed closely by *entailment* and *contradiction*. These types of disagreements are unlikely to result from discrepancies in the labeling guidelines, as they occur in both directions. However, upon closer examination of the samples where disagreements occurred between *entailment* and *neutral*, we observed that it was often difficult to determine whether the information in the hypothesis was newly introduced or could be deduced from the premise. In cases where disagreements occurred between *entailment* and *contradiction*, we found that the premises were often lengthy and contained a significant amount of information, making it challenging to identify the exact location of the contradiction.

### **3.4 Robustness Check**

Take into account both your and your partner's annotations, determine the final labels of the 100 test datapoints, by editing the value of the key "label" in each of your datapoint.

Evaluate the performance of your developed model in 1.4 (still under the first hyperparameter setting) on your annotated 100 test datapoints, and compare with the model performance on the validation set.

> **Question**: Do you think that your developed model has a good robuestness of handling out-of-domain NLI predictions?

In [ ]:
path = f"{ROOT_PATH}/nli_data/12-final.jsonl"
labeled_dataset = NLIDataset(path, tokenizer)
labeled_loss, acc, f1_ent, f1_neu, f1_con = evaluate(
    labeled_dataset, model, device, batch_size, no_labels=False, result_save_file= f"{ROOT_PATH}/predictions/labeled_validation.jsonl")
macro_f1 = (f1_ent + f1_neu + f1_con) / 3
print()
print(f'Validation Loss: {labeled_loss:.3f} | Accuracy: {acc*100:.2f}%')
print(f'F1: ({f1_ent*100:.2f}%, {f1_neu*100:.2f}%, {f1_con*100:.2f}%) | Macro-F1: {macro_f1*100:.2f}%')

Building NLI Dataset...


100it [00:00, 249.22it/s]
Evaluation: 100%|██████████| 7/7 [00:00<00:00, 27.37it/s]

Domain: travel
Validation Loss: 0.915 | Accuracy: 65.00%
F1: (72.09%, 51.06%, 65.67%) | Macro-F1: 62.94%


**Answer** : With the out-domain NLI dataset, we see that the accuracy and the Macro-F1 drop significantly compared to what we saw in point 1.2. 

Our NLP model's performance degradation on out-of-domain samples highlights the fact that we trained only on a small dataset and the importance of diversifying the training data and incorporating a wider range of samples to enhance its generalization capability.

## **Task4: Data Augmentation**

Finally, we consider to use a data augmentation method to create more training data, and use the augmented data to improve the model performance. The data augmentation method we are going to use is [EDA](https://aclanthology.org/D19-1670/).

### **4.1 EDA: Easy Data Augmentation algorithm for Text**

For this section, we will need to implement the most simple data augmentation techniques on textual sentences, including **SR** (Synonym Replacement), **RD** (Random Deletion), **RS** (Random Swap), **RI** (Random Insertion). 

You should complete all the functions in `eda.py` script, and you can test them with a simple testcase by running the following cell.

- **Synonym Replacement (SR)**
> In Synonym Replacement, we randomly replace some words in the sentence with their synonyms.

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
  
from nltk.corpus import wordnet

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mathi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mathi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


You can test whether you get the synonyms right and see an example with synonym replacement.

In [ ]:
from eda import get_synonyms
from testA2 import test_get_synonyms

test_get_synonyms(get_synonyms)

The synonyms for the word "task" are:  ['job', 'undertaking', 'tax', 'labor', 'project', 'chore']


In [ ]:
from eda import synonym_replacement

print(f" Example of Synonym Replacement: {synonym_replacement('hey man how are you doing',3)}")

 Example of Synonym Replacement: hey humanity how are you doing


- **Random Deletion (RD)**

> In Random Deletion, we randomly delete a word if a uniformly generated number between 0 and 1 is smaller than a pre-defined threshold. This allows for a random deletion of some words of the sentence.

In [ ]:
from eda import random_deletion

print(f" Example of Random Deletion: {random_deletion('hey man how are you doing', p=0.3, max_deletion_n=3)}")

 Example of Random Deletion: hey how are you doing


- **Random Swap (RS)**
> In Random Swap, we randomly swap the order of two words in a sentence.

In [ ]:
from eda import swap_word

print(f" Example of Random Swap: {swap_word('hey man how are you doing')}")

 Example of Random Swap: hey man how are doing you


- **Random Insertion (RI)**
> Finally, in Random Insertion, we randomly insert synonyms of a word at a random position.
> Data augmentation operations should not change the true label of a sentence, as that would introduce unnecessary noise into the data. Inserting a synonym of a word in a sentence, opposed to a random word, is more likely to be relevant to the context and retain the original label of the sentence.

In [ ]:
from eda import random_insertion

print(f" Example of Random Insertion: {random_insertion('hey man how are you doing', n=2)}")

 Example of Random Insertion: hey man how are isle of man you human being doing


### **4.2 Augment Your Model**

Combine all the functions you have implemented in 4.1, you can come up with your own data augmentation pipeline with various p and n ;)

Next step is to expand the training data you used in Task1, re-train your model in 1.4 on your augmented data, and re-evaluate its performance on both the given validation set as well as on your manually annotated 100 test datapoints. 

Discuss the improvements that your data augmentation brings to your model. ***Include some examples of old vs. new model predictions to demonstrate the improvements.***

**Warning: In terms of data size and training time control, we stipulate that your augmented training data should not be larger than 100M.** (Currently the training data train.jsonl is about 25M.)

In [ ]:
def aug(sent,n,p):
    print(f" Original Sentence : {sent}")
    print(f" SR Augmented Sentence : {synonym_replacement(sent, n)}")
    print(f" RD Augmented Sentence : {random_deletion(sent, p, n)}")
    print(f" RS Augmented Sentence : {swap_word(sent)}")
    print(f" RI Augmented Sentence : {random_insertion(sent,n)}")
    
aug('hey man how are you doing', p=0.2, n=2)

 Original Sentence : hey man how are you doing
 SR Augmented Sentence : hey mankind how are you doing
 RD Augmented Sentence : hey man how you doing
 RS Augmented Sentence : you man how are hey doing
 RI Augmented Sentence : hey military personnel world man how are you doing


- Augment training dataset and Re-train your model

In [ ]:
import math
p = 0.3
n = 3
def aug_sent(sent,p):
    out = [sent]
    not_used = random.randint(0,3) if len(sent.split()) > 1 else 2
    if not_used != 0 :
        out.append(synonym_replacement(sent, n))
    if not_used != 1 :
        out.append(random_deletion(sent, p, n))
    if not_used != 2 :
        out.append(swap_word(sent))
    if not_used != 3 :
        out.append(random_insertion(sent,n))
    return out

def aug_sample(sample):
    out = []
    aug_pre = aug_sent(sample['premise'],p)
    aug_hyp = aug_sent(sample['hypothesis'],p)
    random.shuffle(aug_hyp)
    for i in range(len(aug_pre)):
        out.append(
        {
            'premise' : aug_pre[i],
            'hypthesis' : aug_hyp[i],
            'label' : sample['label']
        })
    return out

def aug_data(data_repo):
    final = []
    with jsonlines.open(data_repo, "r") as reader:
            for sample in tqdm(reader.iter()):
                final.extend(aug_sample(sample))
    return final
aug_sent('hey man how are you doing', p=0.3)
augmented_data = aug_data(ROOT_PATH+"/nli_data/train.jsonl")

with jsonlines.open(ROOT_PATH+"/nli_data/augmented_train.jsonl", mode='w') as writer:
    for sample in augmented_data:
        writer.write(sample)

In [ ]:
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=3)
model.to(device)

train_dataset = NLIDataset(ROOT_PATH+"/nli_data/augmented_train.jsonl", tokenizer)
dev_dataset = NLIDataset(ROOT_PATH+"/nli_data/dev_in_domain.jsonl", tokenizer)

batch_size = 32
epochs = 4
max_grad_norm = 1.0
warmup_percent = 0.3
model_save_root = ROOT_PATH+'/runs/augmented/'

In [15]:
learning_rate = 8e-6

train(train_dataset, dev_dataset, model, device, batch_size, epochs,
      learning_rate, warmup_percent, max_grad_norm, model_save_root)

Evaluation: 100%|████████████████████████████████████████████████████| 307/307 [00:14<00:00, 21.19it/s]


Epoch: 0 | Training Loss: 0.222 | Validation Loss: 0.901
Epoch 0 NLI Validation:
Accuracy: 76.88% | F1: (79.78%, 72.13%, 78.39%) | Macro-F1: 76.77%
Model Saved!


Evaluation: 100%|████████████████████████████████████████████████████| 307/307 [00:14<00:00, 21.20it/s]


Epoch: 1 | Training Loss: 0.095 | Validation Loss: 1.252
Epoch 1 NLI Validation:
Accuracy: 75.94% | F1: (79.33%, 70.54%, 77.34%) | Macro-F1: 75.74%


Evaluation: 100%|████████████████████████████████████████████████████| 307/307 [00:14<00:00, 21.28it/s]


Epoch: 2 | Training Loss: 0.042 | Validation Loss: 1.678
Epoch 2 NLI Validation:
Accuracy: 76.07% | F1: (79.38%, 71.33%, 77.23%) | Macro-F1: 75.98%


Evaluation: 100%|████████████████████████████████████████████████████| 307/307 [00:14<00:00, 21.23it/s]

Epoch: 3 | Training Loss: 0.027 | Validation Loss: 1.854
Epoch 3 NLI Validation:
Accuracy: 76.04% | F1: (79.20%, 71.25%, 77.31%) | Macro-F1: 75.92%


In [17]:
# Get the resulted with the new model for each of the domains
checkpoint = ROOT_PATH+'/runs/augmented/lr{}-warmup{}'.format(learning_rate, warmup_percent)

tokenizer = DistilBertTokenizer.from_pretrained(checkpoint)
model = DistilBertForSequenceClassification.from_pretrained(checkpoint)
model.to(device)

dev_loss, acc, f1_ent, f1_neu, f1_con = evaluate(
    dev_dataset, model, device, batch_size, no_labels=False, result_save_file= f"{ROOT_PATH}/predictions/augmented/validation_dataset.jsonl")
macro_f1 = (f1_ent + f1_neu + f1_con) / 3

print(f'Validation Dataset')
print(f'Validation Loss: {dev_loss:.3f} | Accuracy: {acc*100:.2f}%')
print(f'F1: ({f1_ent*100:.2f}%, {f1_neu*100:.2f}%, {f1_con*100:.2f}%) | Macro-F1: {macro_f1*100:.2f}%')


path = f"{ROOT_PATH}/nli_data/12-final.jsonl"
labeled_dataset = NLIDataset(path, tokenizer)
labeled_loss, acc, f1_ent, f1_neu, f1_con = evaluate(
    labeled_dataset, model, device, batch_size, no_labels=False, result_save_file= f"{ROOT_PATH}/predictions/augmented/labeled_validation.jsonl")
macro_f1 = (f1_ent + f1_neu + f1_con) / 3
print(f'Labeled dataset')
print(f'Validation Loss: {labeled_loss:.3f} | Accuracy: {acc*100:.2f}%')
print(f'F1: ({f1_ent*100:.2f}%, {f1_neu*100:.2f}%, {f1_con*100:.2f}%) | Macro-F1: {macro_f1*100:.2f}%')


Evaluation: 100%|████████████████████████████████████████████████████| 307/307 [00:14<00:00, 21.87it/s]


Validation Dataset
Validation Loss: 0.901 | Accuracy: 76.88%
F1: (79.78%, 72.13%, 78.39%) | Macro-F1: 76.77%
Building NLI Dataset...


100it [00:00, 1068.70it/s]
Evaluation: 100%|████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 30.59it/s]

Labeled dataset
Validation Loss: 1.291 | Accuracy: 66.00%
F1: (75.29%, 54.17%, 62.69%) | Macro-F1: 64.05%


In [18]:
results = [
    {"dataset" : "validation", "model" : 'base_model', "accuracy" : "77.08%", "macro-f1" : "77.01%"},
    {"dataset" : "validation", "model" : 'augmented_model', "accuracy" : "76.88%", "macro-f1" : "76.77%"},
    {"dataset" : "labeled", "model" : 'base_model', "accuracy" : "65.00%", "macro-f1" : "62.94%"},
    {"dataset" : "labeled", "model" : 'augmented_model', "accuracy" : "66.00%", "macro-f1" : "64.05%"}
]
import pandas as pd
pd.DataFrame(results)

,dataset,model,accuracy,macro-f1
0,validation,base_model,77.08%,77.01%
1,validation,augmented_model,76.88%,76.77%
2,labeled,base_model,65.00%,62.94%
3,labeled,augmented_model,66.00%,64.05%


In [28]:
save_repo_base = f"{ROOT_PATH}/predictions/labeled_validation.jsonl"
save_repo_augment = f"{ROOT_PATH}/predictions/augmented/labeled_validation.jsonl"
predictions = []
with jsonlines.open(save_repo_base, "r") as reader:
    for sample in tqdm(reader.iter()):
        predictions.append(sample)

with jsonlines.open(save_repo_augment, "r") as reader:
    for i, sample in enumerate(reader.iter()):
        predictions[i].update({'prediction_augm' : sample['prediction']})

100it [00:00, 36567.60it/s]


In [52]:
disagreements = [pred for pred in predictions if pred['prediction'] != pred['prediction_augm']]
improvments = [pred for pred in predictions if pred['prediction'] != pred['prediction_augm'] and pred['prediction_augm'] == pred['label']]
reduction = [pred for pred in predictions if pred['prediction'] != pred['prediction_augm'] and pred['prediction'] == pred['label']]


n = 3
data = improvments
print(f"There are {len(data)} label that get imporoved by augmentating the model")
print(f"Here are {n} of them : ")
for i in np.random.randint(0, len(data)-1, size =n) :
    print(f"""
premise : {data[i]['premise']},
hypothesis : {data[i]['hypothesis']},
prediction_base : {data[i]['prediction']},
prediction_augm : {data[i]['prediction_augm']},
label : {data[i]['label']},""")
data = reduction
print(f"\n*********************************************************\n")
print(f"There are {len(data)} label that get worsend by augmentating the model")
print(f"Here are {n} of them : ")
for i in np.random.randint(0, len(data)-1, size =n) :
    print(f"""
premise : {data[i]['premise']},
hypothesis : {data[i]['hypothesis']},
prediction_base : {data[i]['prediction']},
prediction_augm : {data[i]['prediction_augm']},
label : {data[i]['label']},""")

There are 8 label that get imporoved by augmentating the model
Here are 3 of them : 

premise : These monies also help defray the cost of putting on activities for faculty and students such as the Herron Awards Night.,
hypothesis : The money help pay for activities for staff and students.,
prediction_base : contradiction,
prediction_augm : entailment,
label : entailment,

premise : When a task lies at the outer edge of the child's current capabilities, more direct guidance is necessary to bring it within range of mastery.,
hypothesis : If the child can't do the task, guiding them will allow them to potentially master it.,
prediction_base : neutral,
prediction_augm : entailment,
label : entailment,

premise : The data suggest that when a manufacturer chooses the same inventory policy for all products, its order-fulfillment rate for highly variable products is usually worse than for low variation products.,
hypothesis : Manufacturer's with a blanket inventory policy had better order-fulf

**Answer** : Despite using an augmented dataset, our new model has not shown a significant improvement. Instead, we have observed slightly worse results on the validation set, and a slight increase on the labeled dataset. 

This outcome was unexpected, as we had hoped for a significant improvement in performance. One possible explanation for this could be that by augmenting the data, the sentences have been altered, which may have impacted the inference process and led the model to learn incorrect inferences.

In the above you can see 3 sample that get improved by augmenting the dataset and 3 sample that get worsend. What is interesting to see is that sample that are affected badly by the augmentation of the dataset are mostly samples with small sentences (at least in the hypothesis). This might be explained by the fact when we removed words in deletion, we affected the meaning of the sentences and then the small sentences get affected.

The examples provided above demonstrate that augmenting the dataset improved the performance for some samples, while it worsened it for others. Notably, it is interesting to observe that the samples that were negatively impacted by the dataset augmentation were mostly those with shorter sentences (at least in the hypothesis). This observation can be explained by the fact that when we removed words through deletion, the meaning of the sentences may have been altered, which results in worse results for short sentences.

### **5 Upload Your Notebook, Data and Models**

Please **rename** your filled jupyter notebook as **your Sciper number** and upload it to your GitHub Classroom repository, **with all cells run and output results shown**.

**Note:** We are **not** responsible for re-running the cells in your notebook.

Please also submit all your processed (e.g., anotated and augmented) datasets, as well as all your trained models in Task 1 and Task 4, in your GitHub Classroom repository.

The datasets and models that you need to submit include:

**1. The best model checkpoint you trained in the Section 1.2 "Start Training and Validation!"**

**2. The best model prediction results in the Section 1.2 "Fine-Grained Validation"**

**3. Your annotated test dataset in the Section 3.2 "Annotate Your 100 Datapoints with Partner(s)"**

**4. Your augmented training data and best model checkpoint in the Section 4.2 "Augment Your Model"**

**Note:** You may need to use [GitHub LFS](https://edstem.org/eu/courses/379/discussion/27240) for submitting large files.